In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
cd drive/MyDrive/Colab\ Notebooks/scripts

/content/drive/MyDrive/Colab Notebooks/scripts


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import sys
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score
import os
from collections import Counter

In [4]:
from feature_selection import forwardFeatureSelection

from NonLinCFA import NonLinCFA
from aux_GenLinCFA import prepare_target_binary

from aux_NonLinCFA import *
import random

In [5]:
cd ..

/content/drive/MyDrive/Colab Notebooks


# NonLinCFA aggregations standardized target



## Temp Prec

In [ ]:
basins = ['Emiliani1', 'Emiliani2', 'Garda_Mincio', 'Dora', 'Piemonte_Nord', 'Piemonte_Sud']

path_target = "./csv/"
path_features = './features_allvalues/'
destination_folder = './NonLinCFA/temp_prec_internal_ordering/dyn_eps_lin/' # eps = fixed_value/#basin_coord
plots_folder = './NonLinCFA/for_plots/internal_ordering/'

for basin in basins:
  print('####################' + basin + '####################')
  target_df_train, target_df_val, target_df_test, target_df_trainVal = prepare_target('',max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01',
      path=path_target+basin+'.csv', window_size = 1)
  eps = 0.1
  actual_path = path_features+basin+'_aggreg.csv'
  output, aggregate_trainVal, aggregate_test = aggregate_unfolded_data(actual_path,['cyclostationary_mean_tg',
                                                                            'cyclostationary_mean_tg_1w',
                                                                            'cyclostationary_mean_tg_4w',
                                                                            'cyclostationary_mean_tg_8w',
                                                                            'cyclostationary_mean_tg_12w',
                                                                            'cyclostationary_mean_tg_16w',
                                                                            'cyclostationary_mean_tg_24w',
                                                                            'cyclostationary_mean_rr',
                                                                            'cyclostationary_mean_rr_1w',
                                                                            'cyclostationary_mean_rr_4w',
                                                                            'cyclostationary_mean_rr_8w',
                                                                            'cyclostationary_mean_rr_12w',
                                                                            'cyclostationary_mean_rr_16w',
                                                                            'cyclostationary_mean_rr_24w'
                                                                            ],
                                                                      target_df_trainVal, eps=eps,
                                                                      max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01')

  #agg_trainVal_string = plots_folder + basin + "_trainVal_aggreg"
  #agg_test_string = plots_folder + basin + "_test_aggreg"
  #aggregate_trainVal.to_csv(agg_trainVal_string, index = False)
  #aggregate_test.to_csv(agg_test_string, index = False)

  selected_colnames = FS_with_linearWrapper(aggregate_trainVal, target_df_train, target_df_val, min(50,aggregate_trainVal.shape[1]-1), 228)

  print('\nFull model and selected features with wrapper\n')
  compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)

  print('\nFull model and best 5 selected features with wrapper\n')
  compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])

  train_string = destination_folder + basin + '_nonLinCFA_wrapper_best5_train.csv'
  val_string = destination_folder + basin + '_nonLinCFA_wrapper_best5_val.csv'
  test_string = destination_folder + basin + '_nonLinCFA_wrapper_best5_test.csv'
  X_train_wrapper = aggregate_trainVal.loc[:410,selected_colnames[0:5]]
  X_validation_wrapper = aggregate_trainVal.loc[411:,selected_colnames[0:5]]
  X_train_validation_wrapper = pd.concat([X_train_wrapper, X_validation_wrapper])
  X_test_wrapper = aggregate_test.loc[:,selected_colnames[0:5]]
  X_train_wrapper.to_csv(train_string, index=False)
  X_validation_wrapper.to_csv(val_string, index=False)
  X_test_wrapper.to_csv(test_string, index=False)

  res = {
          "delta" : [],
          "numSelected" : [],
          "selectedFeatures" : []
      }

  res['selectedFeatures'] = forwardFeatureSelection(10,np.array(aggregate_trainVal),
                                                    np.array(target_df_trainVal.mean_std),res,10,1)

  selectedFeatures='selectedFeatures'
  print(f'\n{res[selectedFeatures]}\n')

  selected_colnames = aggregate_trainVal.columns[res['selectedFeatures']]

  print('\nFull model and selected features with CMI\n')
  compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)

  print('\nFull model and best 5 selected features with CMI\n')
  compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])

  train_string = destination_folder + basin + '_nonLinCFA_best5_CMI_train.csv'
  val_string = destination_folder + basin + '_nonLinCFA_best5_CMI_val.csv'
  test_string = destination_folder + basin + '_nonLinCFA_best5_CMI_test.csv'

  X_train_CMI5 = aggregate_trainVal.loc[:410,selected_colnames[0:5]]
  X_validation_CMI5 = aggregate_trainVal.loc[411:,selected_colnames[0:5]]
  X_train_validation_CMI5 = pd.concat([X_train_CMI5, X_validation_CMI5])
  X_test_CMI5 = aggregate_test.loc[:,selected_colnames[0:5]]

  selected_colnames_CMI5 = aggregate_trainVal.loc[:,selected_colnames[0:5]].columns.values

  X_train_CMI5.to_csv(train_string, index=False)
  X_validation_CMI5.to_csv(val_string, index=False)
  X_test_CMI5.to_csv(test_string, index=False)

  train_string = destination_folder + basin + '_nonLinCFA_CMI_train.csv'
  val_string = destination_folder + basin + '_nonLinCFA_CMI_val.csv'
  test_string = destination_folder + basin + '_nonLinCFA_CMI_test.csv'

  X_train_CMI = aggregate_trainVal.loc[:410,selected_colnames]
  X_validation_CMI = aggregate_trainVal.loc[411:,selected_colnames]
  X_train_validation_CMI = pd.concat([X_train_CMI, X_validation_CMI])
  X_test_CMI = aggregate_test.loc[:,selected_colnames]

  X_train_CMI.to_csv(train_string, index=False)
  X_validation_CMI.to_csv(val_string, index=False)
  X_test_CMI.to_csv(test_string, index=False)

  print('###### Regression ######')
  lin_regr = LinearRegression()

  # CMI
  lin_regr.fit(X_train_validation_CMI, target_df_trainVal.mean_std)
  print("Train R2 linear regression CMI: ", round(lin_regr.score(X_train_validation_CMI, target_df_trainVal.mean_std),3))
  print("Test R2 linear regression CMI: ", round(lin_regr.score(X_test_CMI, target_df_test.mean_std),3), "\n")

  # CMI best 5
  lin_regr.fit(X_train_validation_CMI5, target_df_trainVal.mean_std)
  print("Train R2 linear regression CMI best 5: ", round(lin_regr.score(X_train_validation_CMI5, target_df_trainVal.mean_std),3))
  print("Test R2 linear regression CMI best 5: ", round(lin_regr.score(X_test_CMI5, target_df_test.mean_std),3), "\n")

  # wrapper
  lin_regr.fit(X_train_validation_wrapper, target_df_trainVal.mean_std)
  print("Train R2 linear regression wrapper: ", round(lin_regr.score(X_train_validation_wrapper, target_df_trainVal.mean_std),3))
  print("Test R2 linear regression wrapper: ", round(lin_regr.score(X_test_wrapper, target_df_test.mean_std),3), "\n")

  print('###### Binary Classification ######')

  target_df_train = target_df_train.apply(lambda x: np.sign(x.mean_std), axis=1)
  target_df_val = target_df_val.apply(lambda x: np.sign(x.mean_std), axis=1)
  target_df_test = target_df_test.apply(lambda x: np.sign(x.mean_std), axis=1)
  target_df_trainVal = target_df_trainVal.apply(lambda x: np.sign(x.mean_std), axis=1)

  log_regr = LogisticRegression(solver='lbfgs', random_state = 42)

  # CMI
  log_regr.fit(X_train_validation_CMI, target_df_trainVal)
  print("Train accuracy logistic regression CMI: ", round(log_regr.score(X_train_validation_CMI, target_df_trainVal),3))
  print("Test accuracy logistic regression CMI: ", round(log_regr.score(X_test_CMI, target_df_test),3), "\n")

  # CMI best 5
  log_regr.fit(X_train_validation_CMI5, target_df_trainVal)
  print("Train accuracy logistic regression CMI best 5: ", round(log_regr.score(X_train_validation_CMI5, target_df_trainVal),3))
  print("Test accuracy logistic regression CMI best 5: ", round(log_regr.score(X_test_CMI5, target_df_test),3), "\n")

  # wrapper
  log_regr.fit(X_train_validation_wrapper, target_df_trainVal)
  print("Train accuracy logistic regression wrapper: ", round(log_regr.score(X_train_validation_wrapper, target_df_trainVal),3))
  print("Test accuracy logistic regression wrapper: ", round(log_regr.score(X_test_wrapper, target_df_test),3), "\n")

  output_string = plots_folder + basin + '_aggregations.npy'
  sel_col_string = plots_folder + basin + '_chosen_features.npy'
  np.save(sel_col_string, selected_colnames_CMI5)
  np.save(output_string, output)

####################Emiliani1####################
Number of features: 172

eps value:  0.0005813953488372093
Number of aggregated features: 7

Number of features: 172

eps value:  0.0005813953488372093
Number of aggregated features: 5

Number of features: 172

eps value:  0.0005813953488372093
Number of aggregated features: 2

Number of features: 172

eps value:  0.0005813953488372093
Number of aggregated features: 6

Number of features: 172

eps value:  0.0005813953488372093
Number of aggregated features: 9

Number of features: 172

eps value:  0.0005813953488372093
Number of aggregated features: 7

Number of features: 172

eps value:  0.0005813953488372093
Number of aggregated features: 2

Number of features: 172

eps value:  0.0005813953488372093
Number of aggregated features: 18

Number of features: 172

eps value:  0.0005813953488372093
Number of aggregated features: 17

Number of features: 172

eps value:  0.0005813953488372093
Number of aggregated features: 10

Number of feature

/usr/local/lib/python3.10/dist-packages/numpy/lib/npyio.py:518: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


####################Emiliani2####################
Number of features: 130

eps value:  0.0007692307692307692
Number of aggregated features: 5

Number of features: 130

eps value:  0.0007692307692307692
Number of aggregated features: 3

Number of features: 130

eps value:  0.0007692307692307692
Number of aggregated features: 3

Number of features: 130

eps value:  0.0007692307692307692
Number of aggregated features: 3

Number of features: 130

eps value:  0.0007692307692307692
Number of aggregated features: 3

Number of features: 130

eps value:  0.0007692307692307692
Number of aggregated features: 4

Number of features: 130

eps value:  0.0007692307692307692
Number of aggregated features: 2

Number of features: 130

eps value:  0.0007692307692307692
Number of aggregated features: 4

Number of features: 130

eps value:  0.0007692307692307692
Number of aggregated features: 8

Number of features: 130

eps value:  0.0007692307692307692
Number of aggregated features: 7

Number of features: 

/usr/local/lib/python3.10/dist-packages/numpy/lib/npyio.py:518: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


####################Garda_Mincio####################
Number of features: 67

eps value:  0.0014925373134328358
Number of aggregated features: 1

Number of features: 67

eps value:  0.0014925373134328358
Number of aggregated features: 3

Number of features: 67

eps value:  0.0014925373134328358
Number of aggregated features: 2

Number of features: 67

eps value:  0.0014925373134328358
Number of aggregated features: 2

Number of features: 67

eps value:  0.0014925373134328358
Number of aggregated features: 1

Number of features: 67

eps value:  0.0014925373134328358
Number of aggregated features: 1

Number of features: 67

eps value:  0.0014925373134328358
Number of aggregated features: 1

Number of features: 67

eps value:  0.0014925373134328358
Number of aggregated features: 4

Number of features: 67

eps value:  0.0014925373134328358
Number of aggregated features: 4

Number of features: 67

eps value:  0.0014925373134328358
Number of aggregated features: 4

Number of features: 67

eps

...

/usr/local/lib/python3.10/dist-packages/numpy/lib/npyio.py:518: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


####################Piemonte_Nord####################
Number of features: 89

eps value:  0.0011235955056179776
Number of aggregated features: 7

Number of features: 89

eps value:  0.0011235955056179776
Number of aggregated features: 5

Number of features: 89

eps value:  0.0011235955056179776
Number of aggregated features: 5

Number of features: 89

eps value:  0.0011235955056179776
Number of aggregated features: 5

Number of features: 89

eps value:  0.0011235955056179776
Number of aggregated features: 1

Number of features: 89

eps value:  0.0011235955056179776
Number of aggregated features: 1

Number of features: 89

eps value:  0.0011235955056179776
Number of aggregated features: 1

Number of features: 89

eps value:  0.0011235955056179776
Number of aggregated features: 2

Number of features: 89

eps value:  0.0011235955056179776
Number of aggregated features: 3

Number of features: 89

eps value:  0.0011235955056179776
Number of aggregated features: 4

Number of features: 89

ep

/usr/local/lib/python3.10/dist-packages/numpy/lib/npyio.py:518: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


####################Piemonte_Sud####################
Number of features: 176

eps value:  0.0005681818181818183
Number of aggregated features: 2

Number of features: 176

eps value:  0.0005681818181818183
Number of aggregated features: 4

Number of features: 176

eps value:  0.0005681818181818183
Number of aggregated features: 2

Number of features: 176

eps value:  0.0005681818181818183
Number of aggregated features: 3

Number of features: 176

eps value:  0.0005681818181818183
Number of aggregated features: 3

Number of features: 176

eps value:  0.0005681818181818183
Number of aggregated features: 3

Number of features: 176

eps value:  0.0005681818181818183
Number of aggregated features: 2

Number of features: 176

eps value:  0.0005681818181818183
Number of aggregated features: 3

Number of features: 176

eps value:  0.0005681818181818183
Number of aggregated features: 7

Number of features: 176

eps value:  0.0005681818181818183
Number of aggregated features: 2

Number of feature

/usr/local/lib/python3.10/dist-packages/numpy/lib/npyio.py:518: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


In [ ]:
basins = ['Adda', 'Lambro_Olona', 'Oglio_Iseo', 'Ticino']

path_target = "./csv/"
path_features = './features_allvalues/'
destination_folder = './NonLinCFA/temp_prec_internal_ordering/dyn_eps_lin/' # eps = fixed_value/#basin_coord
plots_folder = './NonLinCFA/for_plots/internal_ordering/'

for basin in basins:
  selected_colnames_CMI5 = []
  print('####################' + basin + '####################')
  target_df_train, target_df_val, target_df_test, target_df_trainVal = prepare_target('',max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01',
      path=path_target+basin+'.csv', window_size = 1)
  eps = 0.01
  actual_path = path_features+basin+'_aggreg.csv'
  output, aggregate_trainVal, aggregate_test = aggregate_unfolded_data(actual_path,['cyclostationary_mean_tg',
                                                                            'cyclostationary_mean_tg_1w',
                                                                            'cyclostationary_mean_tg_4w',
                                                                            'cyclostationary_mean_tg_8w',
                                                                            'cyclostationary_mean_tg_12w',
                                                                            'cyclostationary_mean_tg_16w',
                                                                            'cyclostationary_mean_tg_24w',
                                                                            'cyclostationary_mean_rr',
                                                                            'cyclostationary_mean_rr_1w',
                                                                            'cyclostationary_mean_rr_4w',
                                                                            'cyclostationary_mean_rr_8w',
                                                                            'cyclostationary_mean_rr_12w',
                                                                            'cyclostationary_mean_rr_16w',
                                                                            'cyclostationary_mean_rr_24w'
                                                                            ],
                                                                      target_df_trainVal, eps=eps,
                                                                      max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01')

  #agg_trainVal_string = plots_folder + basin + "_trainVal_aggreg"
  #agg_test_string = plots_folder + basin + "_test_aggreg"
  #aggregate_trainVal.to_csv(agg_trainVal_string, index = False)
  #aggregate_test.to_csv(agg_test_string, index = False)
  selected_colnames = FS_with_linearWrapper(aggregate_trainVal, target_df_train, target_df_val, min(50,aggregate_trainVal.shape[1]-1), 228)

  print('\nFull model and selected features with wrapper\n')
  compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)

  print('\nFull model and best 5 selected features with wrapper\n')
  compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])

  train_string = destination_folder + basin + '_nonLinCFA_wrapper_best5_train.csv'
  val_string = destination_folder + basin + '_nonLinCFA_wrapper_best5_val.csv'
  test_string = destination_folder + basin + '_nonLinCFA_wrapper_best5_test.csv'
  X_train_wrapper = aggregate_trainVal.loc[:410,selected_colnames[0:5]]
  X_validation_wrapper = aggregate_trainVal.loc[411:,selected_colnames[0:5]]
  X_train_validation_wrapper = pd.concat([X_train_wrapper, X_validation_wrapper])
  X_test_wrapper = aggregate_test.loc[:,selected_colnames[0:5]]
  X_train_wrapper.to_csv(train_string, index=False)
  X_validation_wrapper.to_csv(val_string, index=False)
  X_test_wrapper.to_csv(test_string, index=False)

  res = {
          "delta" : [],
          "numSelected" : [],
          "selectedFeatures" : []
      }

  res['selectedFeatures'] = forwardFeatureSelection(10,np.array(aggregate_trainVal),
                                                    np.array(target_df_trainVal.mean_std),res,10,1)

  selectedFeatures='selectedFeatures'
  print(f'\n{res[selectedFeatures]}\n')

  selected_colnames = aggregate_trainVal.columns[res['selectedFeatures']]

  print('\nFull model and selected features with CMI\n')
  compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)

  print('\nFull model and best 5 selected features with CMI\n')
  compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])

  train_string = destination_folder + basin + '_nonLinCFA_best5_CMI_train.csv'
  val_string = destination_folder + basin + '_nonLinCFA_best5_CMI_val.csv'
  test_string = destination_folder + basin + '_nonLinCFA_best5_CMI_test.csv'

  X_train_CMI5 = aggregate_trainVal.loc[:410,selected_colnames[0:5]]
  X_validation_CMI5 = aggregate_trainVal.loc[411:,selected_colnames[0:5]]
  X_train_validation_CMI5 = pd.concat([X_train_CMI5, X_validation_CMI5])
  X_test_CMI5 = aggregate_test.loc[:,selected_colnames[0:5]]

  selected_colnames_CMI5.append(aggregate_trainVal.loc[:,selected_colnames[0:5]].columns.values)

  X_train_CMI5.to_csv(train_string, index=False)
  X_validation_CMI5.to_csv(val_string, index=False)
  X_test_CMI5.to_csv(test_string, index=False)

  train_string = destination_folder + basin + '_nonLinCFA_CMI_train.csv'
  val_string = destination_folder + basin + '_nonLinCFA_CMI_val.csv'
  test_string = destination_folder + basin + '_nonLinCFA_CMI_test.csv'

  X_train_CMI = aggregate_trainVal.loc[:410,selected_colnames]
  X_validation_CMI = aggregate_trainVal.loc[411:,selected_colnames]
  X_train_validation_CMI = pd.concat([X_train_CMI, X_validation_CMI])
  X_test_CMI = aggregate_test.loc[:,selected_colnames]

  X_train_CMI.to_csv(train_string, index=False)
  X_validation_CMI.to_csv(val_string, index=False)
  X_test_CMI.to_csv(test_string, index=False)

  print('###### Regression ######')
  lin_regr = LinearRegression()

  # CMI
  lin_regr.fit(X_train_validation_CMI, target_df_trainVal.mean_std)
  print("Train R2 linear regression CMI: ", round(lin_regr.score(X_train_validation_CMI, target_df_trainVal.mean_std),3))
  print("Test R2 linear regression CMI: ", round(lin_regr.score(X_test_CMI, target_df_test.mean_std),3), "\n")

  # CMI best 5
  lin_regr.fit(X_train_validation_CMI5, target_df_trainVal.mean_std)
  print("Train R2 linear regression CMI best 5: ", round(lin_regr.score(X_train_validation_CMI5, target_df_trainVal.mean_std),3))
  print("Test R2 linear regression CMI best 5: ", round(lin_regr.score(X_test_CMI5, target_df_test.mean_std),3), "\n")

  # wrapper
  lin_regr.fit(X_train_validation_wrapper, target_df_trainVal.mean_std)
  print("Train R2 linear regression wrapper: ", round(lin_regr.score(X_train_validation_wrapper, target_df_trainVal.mean_std),3))
  print("Test R2 linear regression wrapper: ", round(lin_regr.score(X_test_wrapper, target_df_test.mean_std),3), "\n")

  print('###### Binary Classification ######')

  target_df_train = target_df_train.apply(lambda x: np.sign(x.mean_std), axis=1)
  target_df_val = target_df_val.apply(lambda x: np.sign(x.mean_std), axis=1)
  target_df_test = target_df_test.apply(lambda x: np.sign(x.mean_std), axis=1)
  target_df_trainVal = target_df_trainVal.apply(lambda x: np.sign(x.mean_std), axis=1)

  log_regr = LogisticRegression(solver='lbfgs', random_state = 42)

  # CMI
  log_regr.fit(X_train_validation_CMI, target_df_trainVal)
  print("Train accuracy logistic regression CMI: ", round(log_regr.score(X_train_validation_CMI, target_df_trainVal),3))
  print("Test accuracy logistic regression CMI: ", round(log_regr.score(X_test_CMI, target_df_test),3), "\n")

  # CMI best 5
  log_regr.fit(X_train_validation_CMI5, target_df_trainVal)
  print("Train accuracy logistic regression CMI best 5: ", round(log_regr.score(X_train_validation_CMI5, target_df_trainVal),3))
  print("Test accuracy logistic regression CMI best 5: ", round(log_regr.score(X_test_CMI5, target_df_test),3), "\n")

  # wrapper
  log_regr.fit(X_train_validation_wrapper, target_df_trainVal)
  print("Train accuracy logistic regression wrapper: ", round(log_regr.score(X_train_validation_wrapper, target_df_trainVal),3))
  print("Test accuracy logistic regression wrapper: ", round(log_regr.score(X_test_wrapper, target_df_test),3), "\n")

  output_string = plots_folder + basin + '_aggregations.npy'
  sel_col_string = plots_folder + basin + '_chosen_features.npy'
  np.save(sel_col_string, selected_colnames_CMI5)
  np.save(output_string, output)

####################Adda####################
Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 4

Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 5

Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 4

Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 2

Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 2

Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 1

Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 2

Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 3

Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 2

Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 1

Number of features: 92

e

/usr/local/lib/python3.10/dist-packages/numpy/lib/npyio.py:518: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


####################Lambro_Olona####################
Number of features: 55

eps value:  0.00018181818181818183
Number of aggregated features: 6

Number of features: 55

eps value:  0.00018181818181818183
Number of aggregated features: 6

Number of features: 55

eps value:  0.00018181818181818183
Number of aggregated features: 5

Number of features: 55

eps value:  0.00018181818181818183
Number of aggregated features: 4

Number of features: 55

eps value:  0.00018181818181818183
Number of aggregated features: 3

Number of features: 55

eps value:  0.00018181818181818183
Number of aggregated features: 3

Number of features: 55

eps value:  0.00018181818181818183
Number of aggregated features: 2

Number of features: 55

eps value:  0.00018181818181818183
Number of aggregated features: 4

Number of features: 55

eps value:  0.00018181818181818183
Number of aggregated features: 2

Number of features: 55

eps value:  0.00018181818181818183
Number of aggregated features: 2

Number of feature

/usr/local/lib/python3.10/dist-packages/numpy/lib/npyio.py:518: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


####################Oglio_Iseo####################
Number of features: 74

eps value:  0.00013513513513513514
Number of aggregated features: 3

Number of features: 74

eps value:  0.00013513513513513514
Number of aggregated features: 2

Number of features: 74

eps value:  0.00013513513513513514
Number of aggregated features: 1

Number of features: 74

eps value:  0.00013513513513513514
Number of aggregated features: 1

Number of features: 74

eps value:  0.00013513513513513514
Number of aggregated features: 1

Number of features: 74

eps value:  0.00013513513513513514
Number of aggregated features: 1

Number of features: 74

eps value:  0.00013513513513513514
Number of aggregated features: 1

Number of features: 74

eps value:  0.00013513513513513514
Number of aggregated features: 3

Number of features: 74

eps value:  0.00013513513513513514
Number of aggregated features: 2

Number of features: 74

eps value:  0.00013513513513513514
Number of aggregated features: 1

Number of features:

/usr/local/lib/python3.10/dist-packages/numpy/lib/npyio.py:518: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


####################Ticino####################
Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 3

Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 4

Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 5

Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 3

Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 2

Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 3

Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 2

Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 2

Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 2

Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 1

Number of features: 92


/usr/local/lib/python3.10/dist-packages/numpy/lib/npyio.py:518: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


## Temp Prec Snow

In [ ]:
basins = ['Adda', 'Oglio_Iseo', 'Ticino', 'Dora', 'Piemonte_Nord', 'Piemonte_Sud']
path_target = "./csv/"
path_features='./features_allvalues/'
path_features_snow = './features_allvalues/snow/copernicus/relevant_coords/'

destination_folder = './NonLinCFA/temp_prec_snow_copernicus/relevant_coords/'
plots_folder = './NonLinCFA/for_plots/internal_ordering/snow/'

for basin in basins:

    print('####################' + basin + '####################')

    target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target('',max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01',
                                                                                     path=path_target+basin+'.csv')

    eps = 0.01
    actual_path = path_features+basin+'_aggreg.csv'
    snow_actual_path = path_features_snow+basin+'_aggreg_sd_allCoord.csv'


    output,aggregate_trainVal_temp_prec,aggregate_test_temp_prec = aggregate_unfolded_data(actual_path,['cyclostationary_mean_tg',
                                                                             'cyclostationary_mean_tg_1w',
                                                                             'cyclostationary_mean_tg_4w',
                                                                             'cyclostationary_mean_tg_8w',
                                                                             'cyclostationary_mean_tg_12w',
                                                                             'cyclostationary_mean_tg_16w',
                                                                             'cyclostationary_mean_tg_24w',
                                                                             'cyclostationary_mean_rr',
                                                                             'cyclostationary_mean_rr_1w',
                                                                             'cyclostationary_mean_rr_4w',
                                                                             'cyclostationary_mean_rr_8w',
                                                                             'cyclostationary_mean_rr_12w',
                                                                             'cyclostationary_mean_rr_16w',
                                                                             'cyclostationary_mean_rr_24w'
                                                                            ],
                                                                       target_df_trainVal, eps=eps,
                                                                       max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01')
    if os.path.isfile(snow_actual_path):
      output_snow,aggregate_trainVal_snow,aggregate_test_snow = aggregate_unfolded_data(snow_actual_path,['cyclostationary_mean_HS',
                                                                              'cyclostationary_mean_HS_1w',
                                                                              'cyclostationary_mean_HS_4w',
                                                                              'cyclostationary_mean_HS_8w',
                                                                              'cyclostationary_mean_HS_12w',
                                                                              'cyclostationary_mean_HS_16w',
                                                                              'cyclostationary_mean_HS_24w'
                                                                              ],
                                                                        target_df_trainVal, eps=eps,
                                                                        max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01',
                                                                                        scale = 0.26)


      aggregate_trainVal = pd.concat((aggregate_trainVal_snow,aggregate_trainVal_temp_prec),axis=1)
      aggregate_test = pd.concat((aggregate_test_snow,aggregate_test_temp_prec),axis=1)

    selected_colnames = FS_with_linearWrapper(aggregate_trainVal, target_df_train, target_df_val, min(50,aggregate_trainVal.shape[1]-1), 228)

    print('\nFull model and selected features with wrapper\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)

    print('\nFull model and best 5 selected features with wrapper\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])

    train_string = destination_folder + basin + '_nonLinCFA_wrapper_best5_train_withSnow.csv'
    val_string = destination_folder + basin + '_nonLinCFA_wrapper_best5_val_withSnow.csv'
    test_string = destination_folder + basin + '_nonLinCFA_wrapper_best5_test_withSnow.csv'
    X_train_wrapper = aggregate_trainVal.loc[:410,selected_colnames[0:5]]
    X_validation_wrapper = aggregate_trainVal.loc[411:,selected_colnames[0:5]]
    X_train_validation_wrapper = pd.concat([X_train_wrapper, X_validation_wrapper])
    X_test_wrapper = aggregate_test.loc[:,selected_colnames[0:5]]
    X_train_wrapper.to_csv(train_string, index=False)
    X_validation_wrapper.to_csv(val_string, index=False)
    X_test_wrapper.to_csv(test_string, index=False)

    res = {
            "delta" : [],
            "numSelected" : [],
            "selectedFeatures" : []
        }

    res['selectedFeatures'] = forwardFeatureSelection(10,np.array(aggregate_trainVal),
                                                      np.array(target_df_trainVal.mean_std),res,10,1)

    selectedFeatures='selectedFeatures'
    print(f'\n{res[selectedFeatures]}\n')

    selected_colnames = aggregate_trainVal.columns[res['selectedFeatures']]

    print('\nFull model and selected features with CMI\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)

    print('\nFull model and best 5 selected features with CMI\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])

    train_string = destination_folder + basin + '_nonLinCFA_best5_CMI_train_withSnow.csv'
    val_string = destination_folder + basin + '_nonLinCFA_best5_CMI_val_withSnow.csv'
    test_string = destination_folder + basin + '_nonLinCFA_best5_CMI_test_withSnow.csv'

    X_train_CMI5 = aggregate_trainVal.loc[:410,selected_colnames[0:5]]
    X_validation_CMI5 = aggregate_trainVal.loc[411:,selected_colnames[0:5]]
    X_train_validation_CMI5 = pd.concat([X_train_CMI5, X_validation_CMI5])
    X_test_CMI5 = aggregate_test.loc[:,selected_colnames[0:5]]

    selected_colnames_CMI5 = aggregate_trainVal.loc[:,selected_colnames[0:5]].columns.values

    X_train_CMI5.to_csv(train_string, index=False)
    X_validation_CMI5.to_csv(val_string, index=False)
    X_test_CMI5.to_csv(test_string, index=False)

    train_string = destination_folder + basin + '_nonLinCFA_CMI_train_withSnow.csv'
    val_string = destination_folder + basin + '_nonLinCFA_CMI_val_withSnow.csv'
    test_string = destination_folder + basin + '_nonLinCFA_CMI_test_withSnow.csv'

    X_train_CMI = aggregate_trainVal.loc[:410,selected_colnames]
    X_validation_CMI = aggregate_trainVal.loc[411:,selected_colnames]
    X_train_validation_CMI = pd.concat([X_train_CMI, X_validation_CMI])
    X_test_CMI = aggregate_test.loc[:,selected_colnames]

    X_train_CMI.to_csv(train_string, index=False)
    X_validation_CMI.to_csv(val_string, index=False)
    X_test_CMI.to_csv(test_string, index=False)

    print('###### Regression ######')
    lin_regr = LinearRegression()

    # CMI
    lin_regr.fit(X_train_validation_CMI, target_df_trainVal.mean_std)
    print("Train R2 linear regression CMI: ", round(lin_regr.score(X_train_validation_CMI, target_df_trainVal.mean_std),3))
    print("Test R2 linear regression CMI: ", round(lin_regr.score(X_test_CMI, target_df_test.mean_std),3), "\n")

    # CMI best 5
    lin_regr.fit(X_train_validation_CMI5, target_df_trainVal.mean_std)
    print("Train R2 linear regression CMI best 5: ", round(lin_regr.score(X_train_validation_CMI5, target_df_trainVal.mean_std),3))
    print("Test R2 linear regression CMI best 5: ", round(lin_regr.score(X_test_CMI5, target_df_test.mean_std),3), "\n")

    # wrapper
    lin_regr.fit(X_train_validation_wrapper, target_df_trainVal.mean_std)
    print("Train R2 linear regression wrapper: ", round(lin_regr.score(X_train_validation_wrapper, target_df_trainVal.mean_std),3))
    print("Test R2 linear regression wrapper: ", round(lin_regr.score(X_test_wrapper, target_df_test.mean_std),3), "\n")

    print('###### Binary Classification ######')

    target_df_train = target_df_train.apply(lambda x: np.sign(x.mean_std), axis=1)
    target_df_val = target_df_val.apply(lambda x: np.sign(x.mean_std), axis=1)
    target_df_test = target_df_test.apply(lambda x: np.sign(x.mean_std), axis=1)
    target_df_trainVal = target_df_trainVal.apply(lambda x: np.sign(x.mean_std), axis=1)

    log_regr = LogisticRegression(solver='lbfgs', random_state = 42)

    # CMI
    log_regr.fit(X_train_validation_CMI, target_df_trainVal)
    print("Train accuracy logistic regression CMI: ", round(log_regr.score(X_train_validation_CMI, target_df_trainVal),3))
    print("Test accuracy logistic regression CMI: ", round(log_regr.score(X_test_CMI, target_df_test),3), "\n")

    # CMI best 5
    log_regr.fit(X_train_validation_CMI5, target_df_trainVal)
    print("Train accuracy logistic regression CMI best 5: ", round(log_regr.score(X_train_validation_CMI5, target_df_trainVal),3))
    print("Test accuracy logistic regression CMI best 5: ", round(log_regr.score(X_test_CMI5, target_df_test),3), "\n")

    # wrapper
    log_regr.fit(X_train_validation_wrapper, target_df_trainVal)
    print("Train accuracy logistic regression wrapper: ", round(log_regr.score(X_train_validation_wrapper, target_df_trainVal),3))
    print("Test accuracy logistic regression wrapper: ", round(log_regr.score(X_test_wrapper, target_df_test),3), "\n")

    output_string = plots_folder + basin + '_aggregations.npy'
    sel_col_string = plots_folder + basin + '_chosen_features.npy'
    np.save(sel_col_string, selected_colnames_CMI5)
    np.save(output_string, output)

####################Adda####################
Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 4

Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 5

Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 4

Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 2

Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 2

Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 1

Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 2

Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 3

Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 2

Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 1

Number of features: 92

e

/usr/local/lib/python3.10/dist-packages/numpy/lib/npyio.py:518: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


####################Oglio_Iseo####################
Number of features: 74

eps value:  0.00013513513513513514
Number of aggregated features: 3

Number of features: 74

eps value:  0.00013513513513513514
Number of aggregated features: 2

Number of features: 74

eps value:  0.00013513513513513514
Number of aggregated features: 1

Number of features: 74

eps value:  0.00013513513513513514
Number of aggregated features: 1

Number of features: 74

eps value:  0.00013513513513513514
Number of aggregated features: 1

Number of features: 74

eps value:  0.00013513513513513514
Number of aggregated features: 1

Number of features: 74

eps value:  0.00013513513513513514
Number of aggregated features: 1

Number of features: 74

eps value:  0.00013513513513513514
Number of aggregated features: 3

Number of features: 74

eps value:  0.00013513513513513514
Number of aggregated features: 2

Number of features: 74

eps value:  0.00013513513513513514
Number of aggregated features: 1

Number of features:

/usr/local/lib/python3.10/dist-packages/numpy/lib/npyio.py:518: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


####################Ticino####################
Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 3

Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 4

Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 5

Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 3

Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 2

Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 3

Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 2

Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 2

Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 2

Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 1

Number of features: 92


...

/usr/local/lib/python3.10/dist-packages/numpy/lib/npyio.py:518: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


####################Piemonte_Nord####################
Number of features: 89

eps value:  0.00011235955056179776
Number of aggregated features: 8

Number of features: 89

eps value:  0.00011235955056179776
Number of aggregated features: 5

Number of features: 89

eps value:  0.00011235955056179776
Number of aggregated features: 5

Number of features: 89

eps value:  0.00011235955056179776
Number of aggregated features: 5

Number of features: 89

eps value:  0.00011235955056179776
Number of aggregated features: 1

Number of features: 89

eps value:  0.00011235955056179776
Number of aggregated features: 1

Number of features: 89

eps value:  0.00011235955056179776
Number of aggregated features: 1

Number of features: 89

eps value:  0.00011235955056179776
Number of aggregated features: 5

Number of features: 89

eps value:  0.00011235955056179776
Number of aggregated features: 6

Number of features: 89

eps value:  0.00011235955056179776
Number of aggregated features: 4

Number of featur

/usr/local/lib/python3.10/dist-packages/numpy/lib/npyio.py:518: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


####################Piemonte_Sud####################
Number of features: 176

eps value:  5.681818181818182e-05
Number of aggregated features: 3

Number of features: 176

eps value:  5.681818181818182e-05
Number of aggregated features: 4

Number of features: 176

eps value:  5.681818181818182e-05
Number of aggregated features: 2

Number of features: 176

eps value:  5.681818181818182e-05
Number of aggregated features: 4

Number of features: 176

eps value:  5.681818181818182e-05
Number of aggregated features: 6

Number of features: 176

eps value:  5.681818181818182e-05
Number of aggregated features: 5

Number of features: 176

eps value:  5.681818181818182e-05
Number of aggregated features: 2

Number of features: 176

eps value:  5.681818181818182e-05
Number of aggregated features: 7

Number of features: 176

eps value:  5.681818181818182e-05
Number of aggregated features: 5

Number of features: 176

eps value:  5.681818181818182e-05
Number of aggregated features: 4

Number of feature

/usr/local/lib/python3.10/dist-packages/numpy/lib/npyio.py:518: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


## Temp Prec Snow Lakes

In [ ]:
basins = ['Ticino', 'Oglio_Iseo', 'Adda', 'Lambro_Olona']

path_target = "./csv/"
path_features_snow = './features_allvalues/snow/copernicus/relevant_coords/'
path_features='./features_allvalues/'
path_features_lakes='./lakes/'

destination_folder = './NonLinCFA/temp_prec_snow_lakes_copernicus/'

for basin in basins:
    print('####################' + basin + '####################')

    target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target('',max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01',
                                                                                     path=path_target+basin+'.csv')

    eps = 0.01
    actual_path = path_features+basin+'_aggreg.csv'
    snow_actual_path = path_features_snow+basin+'_aggreg_sd_allCoord.csv'
    lakes_actual_path = path_features_lakes + 'lakes_' + basin + '_with_aggreg.csv'

    output,aggregate_trainVal,aggregate_test = aggregate_unfolded_data(actual_path,['cyclostationary_mean_tg',
                                                                             'cyclostationary_mean_tg_1w',
                                                                             'cyclostationary_mean_tg_4w',
                                                                             'cyclostationary_mean_tg_8w',
                                                                             'cyclostationary_mean_tg_12w',
                                                                             'cyclostationary_mean_tg_16w',
                                                                             'cyclostationary_mean_tg_24w',
                                                                             'cyclostationary_mean_rr',
                                                                             'cyclostationary_mean_rr_1w',
                                                                             'cyclostationary_mean_rr_4w',
                                                                             'cyclostationary_mean_rr_8w',
                                                                             'cyclostationary_mean_rr_12w',
                                                                             'cyclostationary_mean_rr_16w',
                                                                             'cyclostationary_mean_rr_24w'
                                                                            ],
                                                                       target_df_trainVal, eps=eps,
                                                                       max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01')

    if os.path.isfile(snow_actual_path):
      output_snow,aggregate_trainVal_snow,aggregate_test_snow = aggregate_unfolded_data(snow_actual_path,['cyclostationary_mean_HS',
                                                                              'cyclostationary_mean_HS_1w',
                                                                              'cyclostationary_mean_HS_4w',
                                                                              'cyclostationary_mean_HS_8w',
                                                                              'cyclostationary_mean_HS_12w',
                                                                              'cyclostationary_mean_HS_16w',
                                                                              'cyclostationary_mean_HS_24w'
                                                                              ],
                                                                        target_df_trainVal, eps=eps,
                                                                        max_train='2010-01-01', max_val='2015-01-01',
                                                                                        max_test='2020-01-01', scale = 0.26)

      aggregate_trainVal = pd.concat((aggregate_trainVal_snow,aggregate_trainVal),axis=1)
      aggregate_test = pd.concat((aggregate_test_snow,aggregate_test),axis=1)

    if os.path.isfile(lakes_actual_path):
      df_lakes = pd.read_csv(lakes_actual_path)
      df_lakes = df_lakes.drop("Unnamed: 0", axis='columns')

      df_lakes_trainVal = df_lakes.loc[(df_lakes['date'] > '2001-01-01') & (df_lakes['date'] <= '2015-01-01'),:]
      df_lakes_trainVal.reset_index(inplace = True, drop = True)
      df_lakes_trainVal = df_lakes_trainVal.iloc[:,3:]
      df_lakes_test = df_lakes.loc[(df_lakes['date'] > '2015-01-01') & (df_lakes['date'] <= '2020-01-01'),:]
      df_lakes_test.reset_index(inplace = True, drop = True)
      df_lakes_test = df_lakes_test.iloc[:,3:]

      aggregate_trainVal = pd.concat((aggregate_trainVal, df_lakes_trainVal),axis=1)
      aggregate_test = pd.concat((aggregate_test, df_lakes_test),axis=1)

    selected_colnames = FS_with_linearWrapper(aggregate_trainVal, target_df_train, target_df_val, min(50,aggregate_trainVal.shape[1]-1), 228)

    print('\nFull model and selected features with wrapper\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)

    print('\nFull model and best 5 selected features with wrapper\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])

    train_string = destination_folder + basin + '_nonLinCFA_wrapper_best5_train_withSnowLakes.csv'
    val_string = destination_folder + basin + '_nonLinCFA_wrapper_best5_val_withSnowLakes.csv'
    test_string = destination_folder + basin + '_nonLinCFA_wrapper_best5_test_withSnowLakes.csv'
    X_train_wrapper = aggregate_trainVal.loc[:410,selected_colnames[0:5]]
    X_validation_wrapper = aggregate_trainVal.loc[411:,selected_colnames[0:5]]
    X_train_validation_wrapper = pd.concat([X_train_wrapper, X_validation_wrapper])
    X_test_wrapper = aggregate_test.loc[:,selected_colnames[0:5]]
    X_train_wrapper.to_csv(train_string, index=False)
    X_validation_wrapper.to_csv(val_string, index=False)
    X_test_wrapper.to_csv(test_string, index=False)

    res = {
            "delta" : [],
            "numSelected" : [],
            "selectedFeatures" : []
        }

    res['selectedFeatures'] = forwardFeatureSelection(10,np.array(aggregate_trainVal),
                                                      np.array(target_df_trainVal.mean_std),res,10,1)

    selectedFeatures='selectedFeatures'
    print(f'\n{res[selectedFeatures]}\n')

    selected_colnames = aggregate_trainVal.columns[res['selectedFeatures']]

    print('\nFull model and selected features with CMI\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)

    print('\nFull model and best 5 selected features with CMI\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])

    train_string = destination_folder + basin + '_nonLinCFA_best5_CMI_train_withSnowLakes.csv'
    val_string = destination_folder + basin + '_nonLinCFA_best5_CMI_val_withSnowLakes.csv'
    test_string = destination_folder + basin + '_nonLinCFA_best5_CMI_test_withSnowLakes.csv'

    X_train_CMI5 = aggregate_trainVal.loc[:410,selected_colnames[0:5]]
    X_validation_CMI5 = aggregate_trainVal.loc[411:,selected_colnames[0:5]]
    X_train_validation_CMI5 = pd.concat([X_train_CMI5, X_validation_CMI5])
    X_test_CMI5 = aggregate_test.loc[:,selected_colnames[0:5]]

    selected_colnames_CMI5 = aggregate_trainVal.loc[:,selected_colnames[0:5]].columns.values

    X_train_CMI5.to_csv(train_string, index=False)
    X_validation_CMI5.to_csv(val_string, index=False)
    X_test_CMI5.to_csv(test_string, index=False)

    train_string = destination_folder + basin + '_nonLinCFA_CMI_train_withSnowLakes.csv'
    val_string = destination_folder + basin + '_nonLinCFA_CMI_val_withSnowLakes.csv'
    test_string = destination_folder + basin + '_nonLinCFA_CMI_test_withSnowLakes.csv'

    X_train_CMI = aggregate_trainVal.loc[:410,selected_colnames]
    X_validation_CMI = aggregate_trainVal.loc[411:,selected_colnames]
    X_train_validation_CMI = pd.concat([X_train_CMI, X_validation_CMI])
    X_test_CMI = aggregate_test.loc[:,selected_colnames]

    X_train_CMI.to_csv(train_string, index=False)
    X_validation_CMI.to_csv(val_string, index=False)
    X_test_CMI.to_csv(test_string, index=False)

    print('###### Regression ######')
    lin_regr = LinearRegression()

    # CMI
    lin_regr.fit(X_train_validation_CMI, target_df_trainVal.mean_std)
    print("Train R2 linear regression CMI: ", round(lin_regr.score(X_train_validation_CMI, target_df_trainVal.mean_std),3))
    print("Test R2 linear regression CMI: ", round(lin_regr.score(X_test_CMI, target_df_test.mean_std),3), "\n")

    # CMI best 5
    lin_regr.fit(X_train_validation_CMI5, target_df_trainVal.mean_std)
    print("Train R2 linear regression CMI best 5: ", round(lin_regr.score(X_train_validation_CMI5, target_df_trainVal.mean_std),3))
    print("Test R2 linear regression CMI best 5: ", round(lin_regr.score(X_test_CMI5, target_df_test.mean_std),3), "\n")

    # wrapper
    lin_regr.fit(X_train_validation_wrapper, target_df_trainVal.mean_std)
    print("Train R2 linear regression wrapper: ", round(lin_regr.score(X_train_validation_wrapper, target_df_trainVal.mean_std),3))
    print("Test R2 linear regression wrapper: ", round(lin_regr.score(X_test_wrapper, target_df_test.mean_std),3), "\n")

    print('###### Binary Classification ######')

    target_df_train = target_df_train.apply(lambda x: np.sign(x.mean_std), axis=1)
    target_df_val = target_df_val.apply(lambda x: np.sign(x.mean_std), axis=1)
    target_df_test = target_df_test.apply(lambda x: np.sign(x.mean_std), axis=1)
    target_df_trainVal = target_df_trainVal.apply(lambda x: np.sign(x.mean_std), axis=1)

    log_regr = LogisticRegression(solver='lbfgs', random_state = 42)

    # CMI
    log_regr.fit(X_train_validation_CMI, target_df_trainVal)
    print("Train accuracy logistic regression CMI: ", round(log_regr.score(X_train_validation_CMI, target_df_trainVal),3))
    print("Test accuracy logistic regression CMI: ", round(log_regr.score(X_test_CMI, target_df_test),3), "\n")

    # CMI best 5
    log_regr.fit(X_train_validation_CMI5, target_df_trainVal)
    print("Train accuracy logistic regression CMI best 5: ", round(log_regr.score(X_train_validation_CMI5, target_df_trainVal),3))
    print("Test accuracy logistic regression CMI best 5: ", round(log_regr.score(X_test_CMI5, target_df_test),3), "\n")

    # wrapper
    log_regr.fit(X_train_validation_wrapper, target_df_trainVal)
    print("Train accuracy logistic regression wrapper: ", round(log_regr.score(X_train_validation_wrapper, target_df_trainVal),3))
    print("Test accuracy logistic regression wrapper: ", round(log_regr.score(X_test_wrapper, target_df_test),3), "\n")

## Temp Prec Lakes

In [ ]:
basins = ['Adda', 'Ticino', 'Oglio_Iseo']
path_target = "./csv/"
path_features='./features_allvalues/'
path_features_lakes='./lakes/'
destination_folder = './NonLinCFA/temp_prec_lakes/internal_ordering/'

for basin in basins:
    print('####################' + basin + '####################')

    target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target('',max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01', path=path_target+basin+'.csv')

    eps = 0.01
    actual_path = path_features+basin+'_aggreg.csv'
    lakes_actual_path = path_features_lakes + 'lakes_' + basin + '_with_aggreg.csv'

    output,aggregate_trainVal_temp_prec,aggregate_test_temp_prec = aggregate_unfolded_data(actual_path,['cyclostationary_mean_tg',
                                                                             'cyclostationary_mean_tg_1w',
                                                                             'cyclostationary_mean_tg_4w',
                                                                             'cyclostationary_mean_tg_8w',
                                                                             'cyclostationary_mean_tg_12w',
                                                                             'cyclostationary_mean_tg_16w',
                                                                             'cyclostationary_mean_tg_24w',
                                                                             'cyclostationary_mean_rr',
                                                                             'cyclostationary_mean_rr_1w',
                                                                             'cyclostationary_mean_rr_4w',
                                                                             'cyclostationary_mean_rr_8w',
                                                                             'cyclostationary_mean_rr_12w',
                                                                             'cyclostationary_mean_rr_16w',
                                                                             'cyclostationary_mean_rr_24w'
                                                                            ],
                                                                       target_df_trainVal, eps=eps,
                                                                       max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01')

    if os.path.isfile(lakes_actual_path):
      df_lakes = pd.read_csv(lakes_actual_path)

      df_lakes = df_lakes.drop("Unnamed: 0", axis='columns')

      df_lakes_trainVal = df_lakes.loc[(df_lakes['date'] > '2001-01-01') & (df_lakes['date'] <= '2015-01-01'),:]
      df_lakes_trainVal.reset_index(inplace = True, drop = True)
      df_lakes_trainVal = df_lakes_trainVal.iloc[:,3:]
      df_lakes_test = df_lakes.loc[(df_lakes['date'] > '2015-01-01') & (df_lakes['date'] <= '2020-01-01'),:]
      df_lakes_test.reset_index(inplace = True, drop = True)
      df_lakes_test = df_lakes_test.iloc[:,3:]

      aggregate_trainVal = pd.concat((df_lakes_trainVal,aggregate_trainVal_temp_prec),axis=1)
      aggregate_test = pd.concat((df_lakes_test,aggregate_test_temp_prec),axis=1)

    selected_colnames = FS_with_linearWrapper(aggregate_trainVal, target_df_train, target_df_val, min(50,aggregate_trainVal.shape[1]-1), 228)

    print('\nFull model and selected features with wrapper\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)

    print('\nFull model and best 5 selected features with wrapper\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])

    train_string = destination_folder + basin + '_nonLinCFA_wrapper_best5_train_withLakes.csv'
    val_string = destination_folder + basin + '_nonLinCFA_wrapper_best5_val_withLakes.csv'
    test_string = destination_folder + basin + '_nonLinCFA_wrapper_best5_test_withLakes.csv'
    X_train_wrapper = aggregate_trainVal.loc[:410,selected_colnames[0:5]]
    X_validation_wrapper = aggregate_trainVal.loc[411:,selected_colnames[0:5]]
    X_train_validation_wrapper = pd.concat([X_train_wrapper, X_validation_wrapper])
    X_test_wrapper = aggregate_test.loc[:,selected_colnames[0:5]]
    X_train_wrapper.to_csv(train_string, index=False)
    X_validation_wrapper.to_csv(val_string, index=False)
    X_test_wrapper.to_csv(test_string, index=False)

    res = {
            "delta" : [],
            "numSelected" : [],
            "selectedFeatures" : []
        }

    res['selectedFeatures'] = forwardFeatureSelection(10,np.array(aggregate_trainVal),
                                                      np.array(target_df_trainVal.mean_std),res,10,1)

    selectedFeatures='selectedFeatures'
    print(f'\n{res[selectedFeatures]}\n')

    selected_colnames = aggregate_trainVal.columns[res['selectedFeatures']]

    print('\nFull model and selected features with CMI\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)

    print('\nFull model and best 5 selected features with CMI\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])

    train_string = destination_folder + basin + '_nonLinCFA_best5_CMI_train_withLakes.csv'
    val_string = destination_folder + basin + '_nonLinCFA_best5_CMI_val_withLakes.csv'
    test_string = destination_folder + basin + '_nonLinCFA_best5_CMI_test_withLakes.csv'

    X_train_CMI5 = aggregate_trainVal.loc[:410,selected_colnames[0:5]]
    X_validation_CMI5 = aggregate_trainVal.loc[411:,selected_colnames[0:5]]
    X_train_validation_CMI5 = pd.concat([X_train_CMI5, X_validation_CMI5])
    X_test_CMI5 = aggregate_test.loc[:,selected_colnames[0:5]]

    selected_colnames_CMI5 = aggregate_trainVal.loc[:,selected_colnames[0:5]].columns.values

    X_train_CMI5.to_csv(train_string, index=False)
    X_validation_CMI5.to_csv(val_string, index=False)
    X_test_CMI5.to_csv(test_string, index=False)

    train_string = destination_folder + basin + '_nonLinCFA_CMI_train_withLakes.csv'
    val_string = destination_folder + basin + '_nonLinCFA_CMI_val_withLakes.csv'
    test_string = destination_folder + basin + '_nonLinCFA_CMI_test_withLakes.csv'

    X_train_CMI = aggregate_trainVal.loc[:410,selected_colnames]
    X_validation_CMI = aggregate_trainVal.loc[411:,selected_colnames]
    X_train_validation_CMI = pd.concat([X_train_CMI, X_validation_CMI])
    X_test_CMI = aggregate_test.loc[:,selected_colnames]

    X_train_CMI.to_csv(train_string, index=False)
    X_validation_CMI.to_csv(val_string, index=False)
    X_test_CMI.to_csv(test_string, index=False)

    print('###### Regression ######')
    lin_regr = LinearRegression()

    # CMI
    lin_regr.fit(X_train_validation_CMI, target_df_trainVal.mean_std)
    print("Train R2 linear regression CMI: ", round(lin_regr.score(X_train_validation_CMI, target_df_trainVal.mean_std),3))
    print("Test R2 linear regression CMI: ", round(lin_regr.score(X_test_CMI, target_df_test.mean_std),3), "\n")

    # CMI best 5
    lin_regr.fit(X_train_validation_CMI5, target_df_trainVal.mean_std)
    print("Train R2 linear regression CMI best 5: ", round(lin_regr.score(X_train_validation_CMI5, target_df_trainVal.mean_std),3))
    print("Test R2 linear regression CMI best 5: ", round(lin_regr.score(X_test_CMI5, target_df_test.mean_std),3), "\n")

    # wrapper
    lin_regr.fit(X_train_validation_wrapper, target_df_trainVal.mean_std)
    print("Train R2 linear regression wrapper: ", round(lin_regr.score(X_train_validation_wrapper, target_df_trainVal.mean_std),3))
    print("Test R2 linear regression wrapper: ", round(lin_regr.score(X_test_wrapper, target_df_test.mean_std),3), "\n")

    print('###### Binary Classification ######')

    target_df_train = target_df_train.apply(lambda x: np.sign(x.mean_std), axis=1)
    target_df_val = target_df_val.apply(lambda x: np.sign(x.mean_std), axis=1)
    target_df_test = target_df_test.apply(lambda x: np.sign(x.mean_std), axis=1)
    target_df_trainVal = target_df_trainVal.apply(lambda x: np.sign(x.mean_std), axis=1)

    log_regr = LogisticRegression(solver='lbfgs', random_state = 42)

    # CMI
    log_regr.fit(X_train_validation_CMI, target_df_trainVal)
    print("Train accuracy logistic regression CMI: ", round(log_regr.score(X_train_validation_CMI, target_df_trainVal),3))
    print("Test accuracy logistic regression CMI: ", round(log_regr.score(X_test_CMI, target_df_test),3), "\n")

    # CMI best 5
    log_regr.fit(X_train_validation_CMI5, target_df_trainVal)
    print("Train accuracy logistic regression CMI best 5: ", round(log_regr.score(X_train_validation_CMI5, target_df_trainVal),3))
    print("Test accuracy logistic regression CMI best 5: ", round(log_regr.score(X_test_CMI5, target_df_test),3), "\n")

    # wrapper
    log_regr.fit(X_train_validation_wrapper, target_df_trainVal)
    print("Train accuracy logistic regression wrapper: ", round(log_regr.score(X_train_validation_wrapper, target_df_trainVal),3))
    print("Test accuracy logistic regression wrapper: ", round(log_regr.score(X_test_wrapper, target_df_test),3), "\n")

####################Adda####################
Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 4

Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 5

Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 4

Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 2

Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 2

Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 1

Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 2

Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 3

Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 2

Number of features: 92

eps value:  0.00010869565217391305
Number of aggregated features: 1

Number of features: 92

e

## Multi task scores

In [ ]:
# for binary classification

from sklearn.metrics import accuracy_score
def MTL_scores(clust_basins, df_train, df_val, df_test, targets_df_train, targets_df_val, targets_df_test):

    colnames = [x for x in df_train.columns if x.startswith(tuple(clust_basins))]

    clusterdf_train_withClass = pd.DataFrame()
    clusterdf_val_withClass = pd.DataFrame()
    clusterdf_test_withClass = pd.DataFrame()

    for i in range(len(clust_basins)):
        clusterdf_train_withClass = pd.concat((clusterdf_train_withClass,pd.concat((df_train[colnames],pd.DataFrame(1+i*np.ones(len(df_train)),columns=['basin'])),axis=1)),axis=0)
        clusterdf_val_withClass = pd.concat((clusterdf_val_withClass,pd.concat((df_val[colnames],pd.DataFrame(1+i*np.ones(len(df_val)),columns=['basin'])),axis=1)),axis=0)
        clusterdf_test_withClass = pd.concat((clusterdf_test_withClass,pd.concat((df_test[colnames],pd.DataFrame(1+i*np.ones(len(df_test)),columns=['basin'])),axis=1)),axis=0)

    for i in range(len(clust_basins)):
        clusterdf_train_withClass[clust_basins[i]] = clusterdf_train_withClass.apply(lambda x: int(x.basin==i+1),axis=1)
        clusterdf_val_withClass[clust_basins[i]] = clusterdf_val_withClass.apply(lambda x: int(x.basin==i+1),axis=1)
        clusterdf_test_withClass[clust_basins[i]] = clusterdf_test_withClass.apply(lambda x: int(x.basin==i+1),axis=1)

    clusterdf_train_withClass = clusterdf_train_withClass.loc[:,clusterdf_train_withClass.columns != 'basin']
    clusterdf_val_withClass = clusterdf_val_withClass.loc[:,clusterdf_val_withClass.columns != 'basin']
    clusterdf_test_withClass = clusterdf_test_withClass.loc[:,clusterdf_test_withClass.columns != 'basin']

    targets_df_train_unfolded = pd.DataFrame()
    targets_df_val_unfolded = pd.DataFrame()
    targets_df_test_unfolded = pd.DataFrame()

    for basin in clust_basins:
        targets_df_train_unfolded =  pd.concat((targets_df_train_unfolded,targets_df_train[basin]),axis=0)
        targets_df_val_unfolded =  pd.concat((targets_df_val_unfolded,targets_df_val[basin]),axis=0)
        targets_df_test_unfolded =  pd.concat((targets_df_test_unfolded,targets_df_test[basin]),axis=0)
    targets_df_train_unfolded = targets_df_train_unfolded.reset_index(drop=True)
    targets_df_val_unfolded = targets_df_val_unfolded.reset_index(drop=True)
    targets_df_test_unfolded = targets_df_test_unfolded.reset_index(drop=True)

    # same scores changing the solver, some differences changing penalty, some improve with l1
    model_ohe = LogisticRegression(max_iter = 500)
    model_ohe.fit(pd.concat((clusterdf_train_withClass,clusterdf_val_withClass)).values,pd.concat((targets_df_train_unfolded,targets_df_val_unfolded)).values.ravel())

    for basin in clust_basins:
        print(basin)
        res = model_ohe.predict(clusterdf_test_withClass.loc[clusterdf_test_withClass[basin]==1].values)
        print(accuracy_score(targets_df_test[basin].values.ravel(), res))

In [7]:
# for linear regression

def MTL_scores(clust_basins, df_train, df_val, df_test, targets_df_train, targets_df_val, targets_df_test):

    colnames = [x for x in df_train.columns if x.startswith(tuple(clust_basins))]

    clusterdf_train_withClass = pd.DataFrame()
    clusterdf_val_withClass = pd.DataFrame()
    clusterdf_test_withClass = pd.DataFrame()

    for i in range(len(clust_basins)):
        clusterdf_train_withClass = pd.concat((clusterdf_train_withClass,pd.concat((df_train[colnames],pd.DataFrame(1+i*np.ones(len(df_train)),columns=['basin'])),axis=1)),axis=0)
        clusterdf_val_withClass = pd.concat((clusterdf_val_withClass,pd.concat((df_val[colnames],pd.DataFrame(1+i*np.ones(len(df_val)),columns=['basin'])),axis=1)),axis=0)
        clusterdf_test_withClass = pd.concat((clusterdf_test_withClass,pd.concat((df_test[colnames],pd.DataFrame(1+i*np.ones(len(df_test)),columns=['basin'])),axis=1)),axis=0)

    for i in range(len(clust_basins)):
        clusterdf_train_withClass[clust_basins[i]] = clusterdf_train_withClass.apply(lambda x: int(x.basin==i+1),axis=1)
        clusterdf_val_withClass[clust_basins[i]] = clusterdf_val_withClass.apply(lambda x: int(x.basin==i+1),axis=1)
        clusterdf_test_withClass[clust_basins[i]] = clusterdf_test_withClass.apply(lambda x: int(x.basin==i+1),axis=1)

    clusterdf_train_withClass = clusterdf_train_withClass.loc[:,clusterdf_train_withClass.columns != 'basin']
    clusterdf_val_withClass = clusterdf_val_withClass.loc[:,clusterdf_val_withClass.columns != 'basin']
    clusterdf_test_withClass = clusterdf_test_withClass.loc[:,clusterdf_test_withClass.columns != 'basin']

    targets_df_train_unfolded = pd.DataFrame()
    targets_df_val_unfolded = pd.DataFrame()
    targets_df_test_unfolded = pd.DataFrame()

    for basin in clust_basins:
        targets_df_train_unfolded =  pd.concat((targets_df_train_unfolded,targets_df_train[basin]),axis=0)
        targets_df_val_unfolded =  pd.concat((targets_df_val_unfolded,targets_df_val[basin]),axis=0)
        targets_df_test_unfolded =  pd.concat((targets_df_test_unfolded,targets_df_test[basin]),axis=0)
    targets_df_train_unfolded = targets_df_train_unfolded.reset_index(drop=True)
    targets_df_val_unfolded = targets_df_val_unfolded.reset_index(drop=True)
    targets_df_test_unfolded = targets_df_test_unfolded.reset_index(drop=True)

    # same scores changing the solver, some differences changing penalty, some improve with l1
    model_ohe = LinearRegression()
    model_ohe.fit(pd.concat((clusterdf_train_withClass,clusterdf_val_withClass)).values,pd.concat((targets_df_train_unfolded,targets_df_val_unfolded)).values.ravel())

    for basin in clust_basins:
        print(basin)
        res = model_ohe.predict(clusterdf_test_withClass.loc[clusterdf_test_withClass[basin]==1].values)
        print(r2_score(targets_df_test[basin].values.ravel(), res))

In [8]:
### continuous targets
basins = ['Adda','Dora','Emiliani1','Emiliani2','Garda_Mincio','Lambro_Olona','Oglio_Iseo','Piemonte_Nord','Piemonte_Sud','Ticino']
path_targets = "./csv/"
targets_df_train = pd.DataFrame()
targets_df_val = pd.DataFrame()
targets_df_test = pd.DataFrame()
targets_df_trainVal = pd.DataFrame()

for basin in basins:
    target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target('',max_train='2010-01-01', max_val='2015-01-01',
                                                                                     max_test='2020-01-01', path=path_targets+basin+'.csv',
                                                                                     window_size = 1)
    targets_df_train[basin] = target_df_train.mean_std
    targets_df_val[basin] = target_df_val.mean_std
    targets_df_test[basin] = target_df_test.mean_std
    targets_df_trainVal[basin] = target_df_trainVal.mean_std

In [ ]:
### binary targets
basins = ['Adda','Dora','Emiliani1','Emiliani2','Garda_Mincio','Lambro_Olona','Oglio_Iseo','Piemonte_Nord','Piemonte_Sud','Ticino']
path_targets = "./csv/"
targets_df_train = pd.DataFrame()
targets_df_val = pd.DataFrame()
targets_df_test = pd.DataFrame()
targets_df_trainVal = pd.DataFrame()

for basin in basins:
    target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target_binary('',max_train='2010-01-01', max_val='2015-01-01',
                                                                                            max_test='2020-01-01', path=path_targets+basin+'.csv',
                                                                                            threshold = None, nopeaks = False,
                                                                                            window_size = 1)
    targets_df_train[basin] = target_df_train.mean_std
    targets_df_val[basin] = target_df_val.mean_std
    targets_df_test[basin] = target_df_test.mean_std
    targets_df_trainVal[basin] = target_df_trainVal.mean_std

# Chosen Features analysis

In [ ]:
basin = 'Oglio_Iseo'
path_features = './NonLinCFA/temp_prec_snow_lakes_copernicus/'
train_temp = pd.read_csv(path_features+basin+'_nonLinCFA_wrapper_best5_train_withSnowLakes.csv')

print("Features chosen by CMI for ", basin, " \ntaken temp, prec, snow, lakes now: ")
idx_dic = {}
for col in train_temp.columns:
    idx_dic[col] = train_temp.columns.get_loc(col)
print('\n'.join(f'{value}: {key}' for key, value in idx_dic.items()))

# Temp Prec

In [ ]:
basins = ['Adda','Dora','Emiliani1','Emiliani2','Garda_Mincio','Lambro_Olona',
          'Oglio_Iseo','Piemonte_Nord','Piemonte_Sud','Ticino']

### CMI best5 features
path_features = './NonLinCFA/temp_prec_internal_ordering/dyn_eps_lin/'

best5_CMI_fulldf_train = pd.DataFrame()
best5_CMI_fulldf_val = pd.DataFrame()
best5_CMI_fulldf_test = pd.DataFrame()

for basin in basins:
    train_temp = pd.read_csv(path_features+basin+'_nonLinCFA_best5_CMI_train.csv')
    val_temp = pd.read_csv(path_features+basin+'_nonLinCFA_best5_CMI_val.csv')
    test_temp = pd.read_csv(path_features+basin+'_nonLinCFA_best5_CMI_test.csv')
    best5_CMI_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
    best5_CMI_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
    best5_CMI_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp

In [ ]:
basins = ['Adda','Dora','Emiliani1','Emiliani2','Garda_Mincio','Lambro_Olona',
          'Oglio_Iseo','Piemonte_Nord','Piemonte_Sud','Ticino']

### CMI features
path_features = './NonLinCFA/temp_prec_internal_ordering/dyn_eps_lin/'

CMI_fulldf_train = pd.DataFrame()
CMI_fulldf_val = pd.DataFrame()
CMI_fulldf_test = pd.DataFrame()

for basin in basins:
    train_temp = pd.read_csv(path_features+basin+'_nonLinCFA_CMI_train.csv')
    val_temp = pd.read_csv(path_features+basin+'_nonLinCFA_CMI_val.csv')
    test_temp = pd.read_csv(path_features+basin+'_nonLinCFA_CMI_test.csv')
    CMI_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
    CMI_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
    CMI_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp

In [ ]:
basins = ['Adda','Dora','Emiliani1','Emiliani2','Garda_Mincio','Lambro_Olona',
          'Oglio_Iseo','Piemonte_Nord','Piemonte_Sud','Ticino']

### wrapper best5 features
path_features = './NonLinCFA/temp_prec_internal_ordering/dyn_eps_lin/'

best5_wrapper_fulldf_train = pd.DataFrame()
best5_wrapper_fulldf_val = pd.DataFrame()
best5_wrapper_fulldf_test = pd.DataFrame()

for basin in basins:
    train_temp = pd.read_csv(path_features+basin+'_nonLinCFA_wrapper_best5_train.csv')
    val_temp = pd.read_csv(path_features+basin+'_nonLinCFA_wrapper_best5_val.csv')
    test_temp = pd.read_csv(path_features+basin+'_nonLinCFA_wrapper_best5_test.csv')
    best5_wrapper_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
    best5_wrapper_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
    best5_wrapper_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp

# Temp Prec Snow

In [9]:
basins = ['Adda', 'Oglio_Iseo', 'Ticino', 'Dora', 'Piemonte_Nord', 'Piemonte_Sud']

### CMI best5 features
path_features = './NonLinCFA/temp_prec_snow_copernicus/relevant_coords/'

best5_CMI_fulldf_train = pd.DataFrame()
best5_CMI_fulldf_val = pd.DataFrame()
best5_CMI_fulldf_test = pd.DataFrame()

for basin in basins:
    train_temp = pd.read_csv(path_features+basin+'_nonLinCFA_best5_CMI_train_withSnow.csv')
    val_temp = pd.read_csv(path_features+basin+'_nonLinCFA_best5_CMI_val_withSnow.csv')
    test_temp = pd.read_csv(path_features+basin+'_nonLinCFA_best5_CMI_test_withSnow.csv')
    best5_CMI_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
    best5_CMI_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
    best5_CMI_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp

basin = 'Lambro_Olona'
path_features = './NonLinCFA/temp_prec_internal_ordering/dyn_eps_lin/'

train_temp = pd.read_csv(path_features+basin+'_nonLinCFA_best5_CMI_train.csv')
val_temp = pd.read_csv(path_features+basin+'_nonLinCFA_best5_CMI_val.csv')
test_temp = pd.read_csv(path_features+basin+'_nonLinCFA_best5_CMI_test.csv')
best5_CMI_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
best5_CMI_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
best5_CMI_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp

In [10]:
basins = ['Adda', 'Oglio_Iseo', 'Ticino', 'Dora', 'Piemonte_Nord', 'Piemonte_Sud']

### CMI features
path_features = './NonLinCFA/temp_prec_snow_copernicus/relevant_coords/'

CMI_fulldf_train = pd.DataFrame()
CMI_fulldf_val = pd.DataFrame()
CMI_fulldf_test = pd.DataFrame()

for basin in basins:
    train_temp = pd.read_csv(path_features+basin+'_nonLinCFA_CMI_train_withSnow.csv')
    val_temp = pd.read_csv(path_features+basin+'_nonLinCFA_CMI_val_withSnow.csv')
    test_temp = pd.read_csv(path_features+basin+'_nonLinCFA_CMI_test_withSnow.csv')
    CMI_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
    CMI_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
    CMI_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp

basin = 'Lambro_Olona'
path_features = './NonLinCFA/temp_prec_internal_ordering/dyn_eps_lin/'

train_temp = pd.read_csv(path_features+basin+'_nonLinCFA_CMI_train.csv')
val_temp = pd.read_csv(path_features+basin+'_nonLinCFA_CMI_val.csv')
test_temp = pd.read_csv(path_features+basin+'_nonLinCFA_CMI_test.csv')
CMI_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
CMI_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
CMI_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp

In [11]:
basins = ['Adda', 'Oglio_Iseo', 'Ticino', 'Dora', 'Piemonte_Nord', 'Piemonte_Sud']

### wrapper best5 features
path_features = './NonLinCFA/temp_prec_snow_copernicus/relevant_coords/'

best5_wrapper_fulldf_train = pd.DataFrame()
best5_wrapper_fulldf_val = pd.DataFrame()
best5_wrapper_fulldf_test = pd.DataFrame()

for basin in basins:
    train_temp = pd.read_csv(path_features+basin+'_nonLinCFA_wrapper_best5_train_withSnow.csv')
    val_temp = pd.read_csv(path_features+basin+'_nonLinCFA_wrapper_best5_val_withSnow.csv')
    test_temp = pd.read_csv(path_features+basin+'_nonLinCFA_wrapper_best5_test_withSnow.csv')
    best5_wrapper_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
    best5_wrapper_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
    best5_wrapper_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp

basin = 'Lambro_Olona'
path_features = './NonLinCFA/temp_prec_internal_ordering/dyn_eps_lin/'

train_temp = pd.read_csv(path_features+basin+'_nonLinCFA_wrapper_best5_train.csv')
val_temp = pd.read_csv(path_features+basin+'_nonLinCFA_wrapper_best5_val.csv')
test_temp = pd.read_csv(path_features+basin+'_nonLinCFA_wrapper_best5_test.csv')
best5_wrapper_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
best5_wrapper_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
best5_wrapper_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp

# Temp Prec Snow Lakes

In [8]:
basins = ['Ticino', 'Oglio_Iseo', 'Adda', 'Lambro_Olona']

### CMI best5 features
path_features = './NonLinCFA/temp_prec_snow_lakes_copernicus/'

best5_CMI_fulldf_train = pd.DataFrame()
best5_CMI_fulldf_val = pd.DataFrame()
best5_CMI_fulldf_test = pd.DataFrame()

for basin in basins:
    train_temp = pd.read_csv(path_features+basin+'_nonLinCFA_best5_CMI_train_withSnowLakes.csv')
    val_temp = pd.read_csv(path_features+basin+'_nonLinCFA_best5_CMI_val_withSnowLakes.csv')
    test_temp = pd.read_csv(path_features+basin+'_nonLinCFA_best5_CMI_test_withSnowLakes.csv')
    best5_CMI_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
    best5_CMI_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
    best5_CMI_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp

In [9]:
basins = ['Ticino', 'Oglio_Iseo', 'Adda', 'Lambro_Olona']

### CMI features
path_features = './NonLinCFA/temp_prec_snow_lakes_copernicus/'

CMI_fulldf_train = pd.DataFrame()
CMI_fulldf_val = pd.DataFrame()
CMI_fulldf_test = pd.DataFrame()

for basin in basins:
    train_temp = pd.read_csv(path_features+basin+'_nonLinCFA_CMI_train_withSnowLakes.csv')
    val_temp = pd.read_csv(path_features+basin+'_nonLinCFA_CMI_val_withSnowLakes.csv')
    test_temp = pd.read_csv(path_features+basin+'_nonLinCFA_CMI_test_withSnowLakes.csv')
    CMI_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
    CMI_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
    CMI_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp

In [10]:
basins = ['Ticino', 'Oglio_Iseo', 'Adda', 'Lambro_Olona']

### wrapper best5 features
path_features = './NonLinCFA/temp_prec_snow_lakes_copernicus/'

best5_wrapper_fulldf_train = pd.DataFrame()
best5_wrapper_fulldf_val = pd.DataFrame()
best5_wrapper_fulldf_test = pd.DataFrame()

for basin in basins:
    train_temp = pd.read_csv(path_features+basin+'_nonLinCFA_wrapper_best5_train_withSnowLakes.csv')
    val_temp = pd.read_csv(path_features+basin+'_nonLinCFA_wrapper_best5_val_withSnowLakes.csv')
    test_temp = pd.read_csv(path_features+basin+'_nonLinCFA_wrapper_best5_test_withSnowLakes.csv')
    best5_wrapper_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
    best5_wrapper_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
    best5_wrapper_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp

# Temp Prec Lakes

In [27]:
basins = ['Ticino', 'Oglio_Iseo', 'Adda']

### CMI best5 features
path_features = './NonLinCFA/temp_prec_lakes/internal_ordering/'

best5_CMI_fulldf_train = pd.DataFrame()
best5_CMI_fulldf_val = pd.DataFrame()
best5_CMI_fulldf_test = pd.DataFrame()

for basin in basins:
    train_temp = pd.read_csv(path_features+basin+'_nonLinCFA_best5_CMI_train_withLakes.csv')
    val_temp = pd.read_csv(path_features+basin+'_nonLinCFA_best5_CMI_val_withLakes.csv')
    test_temp = pd.read_csv(path_features+basin+'_nonLinCFA_best5_CMI_test_withLakes.csv')
    best5_CMI_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
    best5_CMI_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
    best5_CMI_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp

basin = 'Lambro_Olona'
path_features = './NonLinCFA/temp_prec_internal_ordering/dyn_eps_lin/'

train_temp = pd.read_csv(path_features+basin+'_nonLinCFA_best5_CMI_train.csv')
val_temp = pd.read_csv(path_features+basin+'_nonLinCFA_best5_CMI_val.csv')
test_temp = pd.read_csv(path_features+basin+'_nonLinCFA_best5_CMI_test.csv')
best5_CMI_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
best5_CMI_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
best5_CMI_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp

In [29]:
basins = ['Ticino', 'Oglio_Iseo', 'Adda']

### CMI features
path_features = './NonLinCFA/temp_prec_lakes/internal_ordering/'

CMI_fulldf_train = pd.DataFrame()
CMI_fulldf_val = pd.DataFrame()
CMI_fulldf_test = pd.DataFrame()

for basin in basins:
    train_temp = pd.read_csv(path_features+basin+'_nonLinCFA_CMI_train_withLakes.csv')
    val_temp = pd.read_csv(path_features+basin+'_nonLinCFA_CMI_val_withLakes.csv')
    test_temp = pd.read_csv(path_features+basin+'_nonLinCFA_CMI_test_withLakes.csv')
    CMI_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
    CMI_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
    CMI_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp

basin = 'Lambro_Olona'
path_features = './NonLinCFA/temp_prec_internal_ordering/dyn_eps_lin/'

train_temp = pd.read_csv(path_features+basin+'_nonLinCFA_CMI_train.csv')
val_temp = pd.read_csv(path_features+basin+'_nonLinCFA_CMI_val.csv')
test_temp = pd.read_csv(path_features+basin+'_nonLinCFA_CMI_test.csv')
CMI_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
CMI_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
CMI_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp

In [30]:
basins = ['Ticino', 'Oglio_Iseo', 'Adda']

### wrapper best5 features
path_features = './NonLinCFA/temp_prec_lakes/internal_ordering/'

best5_wrapper_fulldf_train = pd.DataFrame()
best5_wrapper_fulldf_val = pd.DataFrame()
best5_wrapper_fulldf_test = pd.DataFrame()

for basin in basins:
    train_temp = pd.read_csv(path_features+basin+'_nonLinCFA_wrapper_best5_train_withLakes.csv')
    val_temp = pd.read_csv(path_features+basin+'_nonLinCFA_wrapper_best5_val_withLakes.csv')
    test_temp = pd.read_csv(path_features+basin+'_nonLinCFA_wrapper_best5_test_withLakes.csv')
    best5_wrapper_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
    best5_wrapper_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
    best5_wrapper_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp

basin = 'Lambro_Olona'
path_features = './NonLinCFA/temp_prec_internal_ordering/dyn_eps_lin/'

train_temp = pd.read_csv(path_features+basin+'_nonLinCFA_wrapper_best5_train.csv')
val_temp = pd.read_csv(path_features+basin+'_nonLinCFA_wrapper_best5_val.csv')
test_temp = pd.read_csv(path_features+basin+'_nonLinCFA_wrapper_best5_test.csv')
best5_wrapper_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
best5_wrapper_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
best5_wrapper_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp

# MTL scores

## CMI best 5

In [23]:
MTL_scores(clust_basins=['Emiliani1','Emiliani2','Garda_Mincio'], df_train=best5_CMI_fulldf_train, df_val=best5_CMI_fulldf_val, df_test=best5_CMI_fulldf_test, targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)

Emiliani1
-0.014517744669723553
Emiliani2
-0.010037458448500791
Garda_Mincio
-0.07339744154500183


In [24]:
MTL_scores(clust_basins=['Dora','Piemonte_Sud', 'Piemonte_Nord'], df_train=best5_CMI_fulldf_train, df_val=best5_CMI_fulldf_val, df_test=best5_CMI_fulldf_test, targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)

Dora
-0.29205489910850146
Piemonte_Sud
-0.05206902691266557
Piemonte_Nord
0.0585558916397908


In [25]:
MTL_scores(clust_basins=['Adda', 'Lambro_Olona', 'Oglio_Iseo', 'Ticino'], df_train=best5_CMI_fulldf_train, df_val=best5_CMI_fulldf_val, df_test=best5_CMI_fulldf_test, targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)

Adda
0.1804925042258657
Lambro_Olona
0.14283102959093497
Oglio_Iseo
0.18119239504163553
Ticino
0.12752723907741315


## CMI

In [26]:
MTL_scores(clust_basins=['Emiliani1','Emiliani2','Garda_Mincio'], df_train=CMI_fulldf_train, df_val=CMI_fulldf_val, df_test=CMI_fulldf_test, targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)

Emiliani1
-0.014517744669723553
Emiliani2
-0.010037458448500791
Garda_Mincio
-0.07339744154500183


In [27]:
MTL_scores(clust_basins=['Dora','Piemonte_Sud', 'Piemonte_Nord'], df_train=CMI_fulldf_train, df_val=CMI_fulldf_val, df_test=CMI_fulldf_test, targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)

Dora
-0.1587187138885835
Piemonte_Sud
0.06443333124880335
Piemonte_Nord
0.13530625148562625


In [28]:
MTL_scores(clust_basins=['Adda', 'Lambro_Olona', 'Oglio_Iseo', 'Ticino'], df_train=CMI_fulldf_train, df_val=CMI_fulldf_val, df_test=CMI_fulldf_test, targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)

Adda
0.16683530072841646
Lambro_Olona
0.11399602240924211
Oglio_Iseo
0.18608333775983354
Ticino
0.10370336968912319


## Wrapper best 5

In [12]:
MTL_scores(clust_basins=['Emiliani1','Emiliani2','Garda_Mincio'], df_train=best5_wrapper_fulldf_train, df_val=best5_wrapper_fulldf_val, df_test=best5_wrapper_fulldf_test, targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)

Emiliani1
-0.014517744669723553
Emiliani2
-0.010037458448500791
Garda_Mincio
-0.07339744154500183


In [13]:
MTL_scores(clust_basins=['Dora','Piemonte_Sud', 'Piemonte_Nord'], df_train=best5_wrapper_fulldf_train, df_val=best5_wrapper_fulldf_val, df_test=best5_wrapper_fulldf_test, targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)

Dora
-0.06083194307625717
Piemonte_Sud
0.11328609927901678
Piemonte_Nord
0.11238616192636108


In [14]:
MTL_scores(clust_basins=['Adda', 'Lambro_Olona', 'Oglio_Iseo', 'Ticino'], df_train=best5_wrapper_fulldf_train, df_val=best5_wrapper_fulldf_val, df_test=best5_wrapper_fulldf_test, targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)

Adda
0.05382670758417063
Lambro_Olona
-0.024199259090986303
Oglio_Iseo
0.034961311535131046
Ticino
0.015471835019220337
